Data format conversion for MLSTM-FCN
===


---
Input
---

A single file contains all samples and their labels: ***L * (3 + D)***



- 1st col: sample_id
- 2nd col: timestamps
- 3rd col: label
- after the 4th col: mts vector with D dimensions   

---
Output
---

Two array-like variables

- X : array with shape (n_ts, d, sz)
        Sequence data.
- y : array with shape (n_ts, 1)
        Target labels.



In [4]:
import numpy as np
import sys, os
module_path = os.path.abspath(os.path.join('../../../SMATE_MTS'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.UEA_utils import *
#%run ../../utils/PolluScope_utils.ipynb


In [5]:
import sys,threading,subprocess

proc=subprocess.Popen('/bin/sh',stdout=subprocess.PIPE,stdin=subprocess.PIPE,stderr=subprocess.STDOUT)
pout=proc.stdout
pin=proc.stdin

def outLoop():
    running=True
    while(running):
        line=pout.readline().decode(sys.stdout.encoding)
        print(line,end='')
        running='\n' in line
    print('Finished')

threading.Thread(target=outLoop).start()


In [3]:
pin.write(b' ls /ssd/jzuo/SMATE_MTS/Datasets/MTS-others/PolluScope_16_12_2020_speed_complet/all_dimensions \n')
pin.flush()

15_OverSample_X_test.npy
15_OverSample_X_train.npy
15_OverSample_y_test.npy
15_OverSample_y_train.npy
15_X_test.npy
15_X_train.npy
15_y_test.npy
15_y_train.npy
17_OverSample_X_test.npy
17_OverSample_X_train.npy
17_OverSample_y_test.npy
17_OverSample_y_train.npy
17_X_test.npy
17_X_train.npy
17_y_test.npy
17_y_train.npy
18_OverSample_X_test.npy
18_OverSample_X_train.npy
18_OverSample_y_test.npy
18_OverSample_y_train.npy
18_X_test.npy
18_X_train.npy
18_y_test.npy
18_y_train.npy
all_dimensions_Test.csv
all_dimensions_Train.csv
polluscop_speed.zip
X_test.npy
X_train.npy
y_test.npy
y_train.npy


In [4]:
'''=================================================== Prepare PolluScope data ========================================================'''

'''
rep = "/ssd/jzuo/SMATE_MTS/Datasets/MTS-others/PolluScope_16_12_2020_speed_complet/"
ds = "all_dimensions"
ds_train = ds + '/' + ds + "_Train.csv"
ds_test = ds + '/' + ds + "_Test.csv"


X_train, X_test, y_train, y_test = load_datasets(rep, ds_train, ds_test, z_normal = True)
'''

class list is [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
class list (norm) is [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [12]:
'''=================================================== Prepare UEA data ========================================================'''

rep = "../../../Datasets/MTS-UEA/"
ds = "EthanolConcentration"
rep_ds_train = rep + ds + "/output_train/"
rep_ds_test = rep + ds + "/output_test/"
meta_csv = "meta_data.csv"  # the meta data of training/testing set
rep_output = rep_ds_train + "out_results/"  # output results, e.g., training loss, models
os.system("mkdir -p " + rep_output)
sup_ratio = 1

# prepare UEA datasets form 'arff' files
dataset = get_UEA_dataset(rep_ds_train, rep_ds_test, meta_csv, sup_ratio, mode = 'load', split_strategy='EqualSplit')

X_train = dataset['X_train']
y_train = dataset['Y_train']
X_test = dataset['X_test']
y_test = dataset['Y_test']
X_sup = dataset['X_sup']  # 3-D Array: N * L * D
X_unsup = dataset['X_unsup']
y_sup = dataset['Y_sup']  # 1-D Array
y_unsup = dataset['Y_unsup']



class list is ['E35' 'E38' 'E40' 'E45']
total number of samples is 261
total number of samples is 263


In [13]:

X_train = np.transpose(X_train, (0, 2, 1))
X_test = np.transpose(X_test, (0, 2, 1))
NB_CLASS = len(np.unique(y_train))
MAX_TIMESTEPS =X_train.shape[-1]
MAX_NB_VARIABLES = X_train.shape[-2]
# Save the datasets
print("Train dataset : ", X_train.shape, y_train.shape)
print("Test dataset : ", X_test.shape, y_test.shape)
print("Train dataset metrics : ", X_train.mean(), X_train.std())
print("Test dataset : ", X_test.mean(), X_test.std())
print("Nb classes train: ", len(np.unique(y_train)))
print("Nb classes test: ", len(np.unique(y_test)))

np.save(rep + ds + '/X_train.npy', X_train)
np.save(rep + ds + '/y_train.npy', y_train)
np.save(rep + ds + '/X_test.npy', X_test)
np.save(rep + ds + '/y_test.npy', y_test)


Train dataset :  (261, 3, 1751) (261,)
Test dataset :  (263, 3, 1751) (263,)
Train dataset metrics :  0.33776522185413554 0.3550588786133562
Test dataset :  0.3382842365567585 0.35526999693239597
Nb classes train:  4
Nb classes test:  4


In [4]:
def get_mapping_labels(y):
    '''
    Convert the classes in dataset into training labels in Keras [0, nbr_class-1]

    class_array: an array of classes for samples in dataset
    '''
    classes, counts_cl = np.unique(y, return_counts=True)
    print("class list is " + str(classes))

    mapping_c_l = {}  # a mappling between classes and labels
    for idx, c in enumerate(list(classes)):
        mapping_c_l.update({c: idx})
    return mapping_c_l

In [24]:
'''=============================Prepare PolluScope data From '.npy' file ==============================='''

rep = "/ssd/jzuo/SMATE_MTS/Datasets/MTS-others/PolluScope_16_12_2020_speed_complet/"
ds = "all_dimensions"
threshold_taux = 0.017
X_train = np.load(rep + ds + '/' + str(int(threshold_taux*1000)) + '_OverSample_'+ 'X_train.npy') #"_OverSample_"
y_train = np.load(rep + ds + '/' + str(int(threshold_taux*1000)) + '_OverSample_'+ 'y_train.npy')
X_test = np.load(rep + ds + '/' + str(int(threshold_taux*1000)) + '_OverSample_'+ 'X_test.npy')
y_test = np.load(rep + ds + '/' + str(int(threshold_taux*1000)) + '_OverSample_'+ 'y_test.npy')
X_train = np.transpose(X_train, (0, 2, 1))
X_test = np.transpose(X_test, (0, 2, 1))

'''********* Remove *speed* dimension *********'''
#X_train = X_train[:, :-1, :]
#X_test = X_test[:, :-1, :]

NB_CLASS = len(np.unique(y_train))
MAX_TIMESTEPS =X_train.shape[-1]
MAX_NB_VARIABLES = X_train.shape[-2] 

if np.max(y_train) != len(y_train) - 1 :
    mapping_c_l = get_mapping_labels(y_train)
    y_train = np.array([mapping_c_l[i] for i in y_train])
    y_test = np.array([mapping_c_l[i] for i in y_test])

print("Train dataset : ", X_train.shape, y_train.shape)
print("Test dataset : ", X_test.shape, y_test.shape)
print("Train dataset metrics : ", X_train.mean(), X_train.std())
print("Test dataset : ", X_test.mean(), X_test.std())
print("Nb classes : ", len(np.unique(y_train)))
print("Nb classes train: ", len(np.unique(y_train)))
print("Nb classes test: ", len(np.unique(y_test)))

np.save(rep + ds + '/X_train.npy', X_train)
np.save(rep + ds + '/y_train.npy', y_train)
np.save(rep + ds + '/X_test.npy', X_test)
np.save(rep + ds + '/y_test.npy', y_test)

class list is [ 1.  2.  3.  4.  5.  7.  8. 10.]
Train dataset :  (481, 8, 10) (481,)
Test dataset :  (527, 8, 10) (527,)
Train dataset metrics :  201.29303546119087 833.440601611844
Test dataset :  201.43323874758403 1104.3744524147357
Nb classes :  8
Nb classes train:  8
Nb classes test:  8


In [14]:
# the labels should range from 0 to nbr_class
print(np.unique(y_train), '\n', np.unique(y_test))

[0. 1. 2. 3.] 
 [0. 1. 2. 3.]


---
Build and train the Network Model
===


In [15]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

from utils_mlstm.keras_utils import train_model, evaluate_model, set_trainable
from utils_mlstm.layer_utils import AttentionLSTM

TRAINABLE = True

Using TensorFlow backend.


In [16]:
def generate_model():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))

    x = Masking()(ip)
    x = LSTM(128)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model


def generate_model_2():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))
    # stride = 10

    # x = Permute((2, 1))(ip)
    # x = Conv1D(MAX_NB_VARIABLES // stride, 8, strides=stride, padding='same', activation='relu', use_bias=False,
    #            kernel_initializer='he_uniform')(x)  # (None, variables / stride, timesteps)
    # x = Permute((2, 1))(x)

    #ip1 = K.reshape(ip,shape=(MAX_TIMESTEPS,MAX_NB_VARIABLES))
    #x = Permute((2, 1))(ip)
    x = Masking()(ip)
    x = AttentionLSTM(128)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model



def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor

    Returns: a keras tensor
    '''
    filters = input._keras_shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

In [17]:

model = generate_model_2()

DATASET_INDEX = rep + ds + '/'



W0106 02:57:36.877216 23335415461696 deprecation_wrapper.py:119] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0106 02:57:37.818396 23335415461696 deprecation.py:323] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0106 02:57:37.853159 23335415461696 deprecation.py:506] From /linkhome/rech/genvsq01/ulz67kb/.conda/envs/SMAT_ADE/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future ver

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 1751)      0                                            
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 1751, 3)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1751, 128)    3200        permute_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 1751, 128)    512         conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation

In [ ]:
start = time.time()
train_model(model, DATASET_INDEX, dataset_prefix=ds+'_', epochs=300, batch_size=128)
print("Training time of %d epochs is %d " %(300, time.time() - start))

Finished processing train dataset..
Finished loading test dataset..

Number of train samples :  261 Number of test samples :  263
Number of classes :  4
Sequence length :  1751
X_train.shape is  (261, 3, 1751)
X_test.shape is  (263, 3, 1751)
Class weights :  [1.00384615 1.00384615 0.98863636 1.00384615]
Train on 261 samples, validate on 263 samples
Epoch 1/300
 - 4s - loss: 1.0462 - acc: 0.5517 - val_loss: 2.2334 - val_acc: 0.2548

Epoch 00001: val_acc improved from -inf to 0.25475, saving model to ./weights/EthanolConcentration__weights.h5
Epoch 2/300
 - 0s - loss: 1.0089 - acc: 0.5747 - val_loss: 1.9680 - val_acc: 0.2433

Epoch 00002: val_acc did not improve from 0.25475
Epoch 3/300
 - 0s - loss: 1.0295 - acc: 0.5556 - val_loss: 1.7834 - val_acc: 0.2624

Epoch 00003: val_acc improved from 0.25475 to 0.26236, saving model to ./weights/EthanolConcentration__weights.h5
Epoch 4/300
 - 0s - loss: 1.0666 - acc: 0.5287 - val_loss: 1.6657 - val_acc: 0.2700

Epoch 00004: val_acc improved from

 - 0s - loss: 1.0442 - acc: 0.5364 - val_loss: 1.9663 - val_acc: 0.2700

Epoch 00056: val_acc did not improve from 0.30798
Epoch 57/300
 - 0s - loss: 1.0392 - acc: 0.5057 - val_loss: 2.0061 - val_acc: 0.2624

Epoch 00057: val_acc did not improve from 0.30798
Epoch 58/300
 - 0s - loss: 1.0461 - acc: 0.5326 - val_loss: 2.1176 - val_acc: 0.2586

Epoch 00058: val_acc did not improve from 0.30798
Epoch 59/300
 - 0s - loss: 1.0268 - acc: 0.5517 - val_loss: 2.5595 - val_acc: 0.2738

Epoch 00059: val_acc did not improve from 0.30798
Epoch 60/300
 - 0s - loss: 1.0128 - acc: 0.5977 - val_loss: 2.7086 - val_acc: 0.2738

Epoch 00060: val_acc did not improve from 0.30798
Epoch 61/300
 - 0s - loss: 1.0606 - acc: 0.5211 - val_loss: 2.5420 - val_acc: 0.2243

Epoch 00061: val_acc did not improve from 0.30798
Epoch 62/300
 - 0s - loss: 1.0848 - acc: 0.5019 - val_loss: 2.7342 - val_acc: 0.2205

Epoch 00062: val_acc did not improve from 0.30798
Epoch 63/300
 - 0s - loss: 1.0479 - acc: 0.5211 - val_loss: 3


Epoch 00116: val_acc did not improve from 0.30798
Epoch 117/300
 - 0s - loss: 0.9914 - acc: 0.5632 - val_loss: 1.9819 - val_acc: 0.2091

Epoch 00117: val_acc did not improve from 0.30798
Epoch 118/300
 - 0s - loss: 1.0084 - acc: 0.5709 - val_loss: 2.2021 - val_acc: 0.2662

Epoch 00118: val_acc did not improve from 0.30798
Epoch 119/300
 - 0s - loss: 1.0089 - acc: 0.5594 - val_loss: 2.5424 - val_acc: 0.2662

Epoch 00119: val_acc did not improve from 0.30798
Epoch 120/300
 - 0s - loss: 0.9849 - acc: 0.5670 - val_loss: 2.9924 - val_acc: 0.2738

Epoch 00120: val_acc did not improve from 0.30798
Epoch 121/300
 - 0s - loss: 0.9830 - acc: 0.5670 - val_loss: 3.2662 - val_acc: 0.2662

Epoch 00121: val_acc did not improve from 0.30798
Epoch 122/300
 - 0s - loss: 0.9783 - acc: 0.6015 - val_loss: 2.9589 - val_acc: 0.2738

Epoch 00122: val_acc did not improve from 0.30798
Epoch 123/300
 - 0s - loss: 1.0205 - acc: 0.5632 - val_loss: 2.8703 - val_acc: 0.3004

Epoch 00123: val_acc did not improve fro


Epoch 00175: val_acc did not improve from 0.32700
Epoch 176/300
 - 0s - loss: 0.9740 - acc: 0.5977 - val_loss: 2.6183 - val_acc: 0.3004

Epoch 00176: val_acc did not improve from 0.32700
Epoch 177/300
 - 0s - loss: 0.9680 - acc: 0.5747 - val_loss: 3.0973 - val_acc: 0.2662

Epoch 00177: val_acc did not improve from 0.32700
Epoch 178/300
 - 0s - loss: 0.9852 - acc: 0.5632 - val_loss: 3.1154 - val_acc: 0.2662

Epoch 00178: val_acc did not improve from 0.32700
Epoch 179/300
 - 0s - loss: 1.0222 - acc: 0.5939 - val_loss: 2.9420 - val_acc: 0.2700

Epoch 00179: val_acc did not improve from 0.32700
Epoch 180/300
 - 0s - loss: 1.0031 - acc: 0.5670 - val_loss: 2.7535 - val_acc: 0.2776

Epoch 00180: val_acc did not improve from 0.32700
Epoch 181/300
 - 0s - loss: 0.9943 - acc: 0.5364 - val_loss: 2.6207 - val_acc: 0.2700

Epoch 00181: val_acc did not improve from 0.32700
Epoch 182/300
 - 0s - loss: 0.9739 - acc: 0.5747 - val_loss: 2.4370 - val_acc: 0.2624

Epoch 00182: val_acc did not improve fro


Epoch 00235: val_acc did not improve from 0.32700
Epoch 236/300
 - 0s - loss: 0.9210 - acc: 0.6207 - val_loss: 1.8697 - val_acc: 0.3042

Epoch 00236: val_acc did not improve from 0.32700
Epoch 237/300
 - 0s - loss: 0.9100 - acc: 0.6322 - val_loss: 1.8701 - val_acc: 0.2586

Epoch 00237: val_acc did not improve from 0.32700
Epoch 238/300
 - 0s - loss: 0.8979 - acc: 0.6552 - val_loss: 1.9987 - val_acc: 0.2319

Epoch 00238: val_acc did not improve from 0.32700
Epoch 239/300
 - 0s - loss: 0.9109 - acc: 0.6284 - val_loss: 2.2041 - val_acc: 0.2395

Epoch 00239: val_acc did not improve from 0.32700
Epoch 240/300
 - 0s - loss: 0.9351 - acc: 0.5862 - val_loss: 2.4800 - val_acc: 0.2662

Epoch 00240: val_acc did not improve from 0.32700
Epoch 241/300
 - 0s - loss: 0.9560 - acc: 0.5939 - val_loss: 2.3193 - val_acc: 0.2852

Epoch 00241: val_acc did not improve from 0.32700
Epoch 242/300
 - 0s - loss: 0.9697 - acc: 0.5670 - val_loss: 2.0476 - val_acc: 0.2890

Epoch 00242: val_acc did not improve fro

In [29]:
print("[MLSTM-FCN] Results on dataset WITH Speed (8 classes): \n Class_sample_threshold=1.7%")
evaluate_model(model, DATASET_INDEX, dataset_prefix=ds+'_', batch_size=128)

[MLSTM-FCN] Results on dataset WITH Speed (8 classes): 
 Class_sample_threshold=1.7%
Finished processing train dataset..
Finished loading test dataset..

Number of train samples :  481 Number of test samples :  527
Number of classes :  8
Sequence length :  10
X_train.shape is  (481, 8, 10)
X_test.shape is  (527, 8, 10)

Evaluating : 
              precision    recall  f1-score   support

           0       0.43      0.54      0.48        52
           1       0.41      0.65      0.50        20
           2       0.90      0.88      0.89       129
           3       0.75      0.75      0.75         8
           4       0.80      0.80      0.80       132
           5       0.80      0.60      0.69       151
           6       0.38      0.48      0.42        25
           7       0.22      0.40      0.29        10

    accuracy                           0.71       527
   macro avg       0.59      0.64      0.60       527
weighted avg       0.74      0.71      0.72       527

Final Accurac

(0.7077798869397202, 1.5153405508008808)

In [ ]:
import sklearn
print(sklearn.__version__)